In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output,State
import plotly.graph_objs as go
import pandas as pd
from datetime import datetime
import pandas_datareader.data as web

/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [2]:
app = dash.Dash()

In [3]:
nsdq = pd.read_csv('NASDAQcompanylist.csv')

In [4]:
nsdq.head()

,Symbol,Name,LastSale,MarketCap,IPOyear,Sector,Industry
0,ABMD,"ABIOMED, Inc.",286.22,1.267320e+10,NaN,Health Care,Medical/Dental Instruments
1,ATVI,"Activision Blizzard, Inc",64.56,4.897726e+10,NaN,Technology,Computer Software: Prepackaged Software
2,ADBE,Adobe Systems Incorporated,218.61,1.076590e+11,1986.0,Technology,Computer Software: Prepackaged Software
3,AMD,"Advanced Micro Devices, Inc.",9.61,9.313356e+09,NaN,Technology,Semiconductors
4,AGNC,AGNC Investment Corp.,18.95,7.415454e+09,2008.0,Consumer Services,Real Estate Investment Trusts


In [5]:
options = []
for x in nsdq.index:
    options.append({'label':nsdq['Name'][x],'value':nsdq['Symbol'][x]})

In [6]:
app.layout = html.Div([
    html.H1('Stock Ticker Dashboard'),
    html.Div([
        html.H3('Select stock symbols:'),
        dcc.Dropdown(
            id='dropdown',
            options=options,
            value=['TSLA'],
            multi=True
        )  
    ],style={'display':'inline-block','vertical-align':'top','padding':'15px','width':'30%'}),
    html.Div([
        html.H3('Select stock symbols:'),
        dcc.DatePickerRange(
            id='datepicker',
            min_date_allowed=datetime(2015, 1, 1),
            max_date_allowed=datetime.today(),
            start_date=datetime(2018, 1, 1),
            end_date=datetime.today()
        )  
    ],style={'display':'inline-block','vertical-align':'top','padding':'15px'}),
    html.Div([
        html.Button(
            id='submit-button',
            n_clicks=0,
            children='Submit',
            style={'fontSize':24, 'marginLeft':'30px'}
        ),
    ], style={'display':'inline-block','padding':'15px'}),
    dcc.Graph(id='stock-graph')
])

In [7]:
@app.callback(Output('stock-graph','figure'),[Input('submit-button','n_clicks')],
             [State('dropdown','value'),State('datepicker','start_date'),State('datepicker','end_date')])
def update_graph(n_clicks,stockticker,start_date,end_date):
    start = datetime.strptime(start_date[:10],'%Y-%m-%d')
    end = datetime.strptime(end_date[:10],'%Y-%m-%d')
    
    traces = []
    for tick in stockticker:
        df = web.get_data_tiingo(tick, start, end, api_key="1af3148b237a675bc6ee5a56102b2e37de46f194")
        traces.append({'x':df.index.get_level_values(1),'y':df.close,'name':tick})
    
    fig = {
        'data':traces,
        'layout':{'title':'Graph'}
    }
    return fig

In [8]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Feb/2021 04:11:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2021 04:11:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2021 04:11:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2021 04:11:53] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2021 04:12:15] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2021 04:12:37] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [05/Feb/2021 04:13:21] "POST /_dash-update-component HTTP/1.1" 200 -
